## Import Libraries

In [34]:
import pandas as pd
import requests
from requests.api import head
import csv
from tqdm import tqdm
import time
import datetime
import urllib.request
import urllib.parse
import os
import glob

## Import Dataset

For this version, since some of the stores were missing placekeys, and thus geocodes, the zip codes were used instead of coordinates.

In [2]:
df_source = pd.read_csv('/home/rishis/cs/HBS/visualcrossing-dataset/data/addresses.csv')

## Create Template for New Dataset

First, prepare the data by creating a dictionary of all zip codes.

zipcodes_dict[skey] = zipcode 

city_dict[skey] = city

state_dict[skey] = state code

In [3]:
zipcodes_dict = {}
city_dict = {}
state_dict = {}


# make dictionary of all cities
for i in tqdm(range(len(df_source))):
    curr_skey = str(df_source.loc[i, "store_skey"]) 
    zipcodes_dict[curr_skey] = str(df_source.loc[i, "zip_code"])
    city_dict[curr_skey] = str(df_source.loc[i, "city"])
    state_dict[curr_skey] = str(df_source.loc[i, "state_code"])

100%|████████████████████████████████████| 1037/1037 [00:00<00:00, 21331.29it/s]


In [4]:
for skey in tqdm(city_dict):
    try:
        url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/' + urllib.parse.quote(str(city_dict[skey])) + '%2C' + urllib.parse.quote(str(state_dict[skey])) + '/2022-05-10/2022-09-30?unitGroup=metric&include=days&key=3FWLMZ9MF7M7QBH73WCBXT2QF&contentType=csv'
        req = urllib.request.Request(url)
        res = urllib.request.urlopen(req)
        
        with open('./data/historical-weather/' + str(skey) + '-historical_weather.csv',"ab") as f:
            f.write(res.read())
        
        df = pd.read_csv('./data/historical-weather/' + str(skey) + '-historical_weather.csv')
        for i in range(len(df)):
            df.loc[i, "skey"] = skey
        df.to_csv('./data/historical-weather/' + str(skey) + '-historical_weather.csv')
    
    except:
        print("Error with", city_dict[skey])

 90%|███████████████████████████████████▉    | 931/1037 [08:41<00:46,  2.26it/s]

Error with Test


100%|███████████████████████████████████████| 1037/1037 [09:50<00:00,  1.76it/s]


In [38]:
os.chdir('/home/rishis/cs/HBS/visualcrossing-dataset/data/historical-weather')
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])

combined_csv.to_csv('../historical_data.csv', index=False, encoding='utf-8-sig')

In [32]:
files = os.listdir('/home/rishis/cs/HBS/visualcrossing-dataset/data/historical-weather')
fileDict = {}

buildDF = pd.read_csv('/home/rishis/cs/HBS/visualcrossing-dataset/data/historical-weather/' + str(files[0]))

for i in range(1, len(files)):
    newDF = pd.read_csv('/home/rishis/cs/HBS/visualcrossing-dataset/data/historical-weather/' + str(files[i]))
    buildDF = pd.concat(buildDF, newDF)
buildDF.head()

/tmp/ipykernel_5466/2565252444.py:8: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  buildDF = pd.concat(buildDF, newDF)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
for skey in tqdm(zipcodes_dict):
    curr_date = datetime.date(2018, 5, 10)
    while curr_date <= datetime.date(2022, 9, 30):
        try:
            start_date = curr_date
            final_date = start_date + datetime.timedelta(days=15)
            start_date_string = str(start_date)
            final_date_string = str(final_date)
            
            json_data = requests.get("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/" + str(zipcodes_dict[str(skey)]) + "/" + start_date_string + "/" + final_date_string + "?unitGroup=us&key=3FWLMZ9MF7M7QBH73WCBXT2QF&contentType=json&forecastBasisDate=" + start_date_string).json()
            df = pd.DataFrame(json_data["days"])
            df['store_skey'] = skey
            df.to_csv('./data/forecast_weather.csv', index=False, mode='a')
            curr_date = final_date + datetime.timedelta(days=1)
        except:
            curr_date = curr_date + datetime.timedelta(days=1)
            print('Errors with', str(skey), 'on', str(curr_date))

In [ ]:
url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/' + 'Lecanto' + '%2C' + 'FL' + '/2022-05-10/2022-09-30?unitGroup=metric&include=days&key=3FWLMZ9MF7M7QBH73WCBXT2QF&contentType=csv'
req = urllib.request.Request(url)
res = urllib.request.urlopen(req)
tracker = 0

with open('./data/historical_weather.csv',"ab") as f:
    f.write(res.read())
        
df = pd.read_csv('./data/historical_weather.csv')
for i in range(tracker, len(df)):
    df.loc[i, "skey"] = 1122 
    tracker = i
tracker += 1
df.to_csv('./data/historical_weather.csv')

In [43]:
df = pd.read_csv('./data/historical_data.csv')
df.head()

,Unnamed: 0,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,...,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations,skey
0,0,"Mt Laurel, NJ, United States",2022-05-10,22.2,9.4,15.9,22.2,7.0,15.7,-3.3,...,10,10,2022-05-10T05:49:24,2022-05-10T20:03:13,0.32,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,"KVAY,KWRI,72408594732,72407493780,F5135,724096...",483
1,1,"Mt Laurel, NJ, United States",2022-05-11,22.2,10.6,16.0,22.2,10.6,16.0,5.4,...,4,10,2022-05-11T05:48:22,2022-05-11T20:04:11,0.37,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KVAY,KWRI,72408594732,72407493780,F5135,724096...",483
2,2,"Mt Laurel, NJ, United States",2022-05-12,23.4,11.2,17.2,23.4,11.2,17.2,13.3,...,7,10,2022-05-12T05:47:22,2022-05-12T20:05:09,0.41,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KVAY,KWRI,72408594732,72407493780,F5135,724096...",483
3,3,"Mt Laurel, NJ, United States",2022-05-13,22.0,15.0,18.0,22.0,15.0,18.0,16.5,...,4,10,2022-05-13T05:46:23,2022-05-13T20:06:06,0.45,"Rain, Overcast",Cloudy skies throughout the day with afternoon...,rain,"KVAY,KWRI,72408594732,72407493780,F5135,724096...",483
4,4,"Mt Laurel, NJ, United States",2022-05-14,20.0,17.2,18.4,20.0,17.2,18.4,17.6,...,2,10,2022-05-14T05:45:26,2022-05-14T20:07:03,0.48,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"KVAY,KWRI,72408594732,72407493780,F5135,724096...",483


In [49]:
skeyDict = {}
for i in range(len(df)):
    skeyDict[df.loc[i, "skey"]] = 0
    
print(skeyDict[211])

for key in zipcodes_dict:
    if key not in skeyDict:
        print(key)
    


0
1061
215
498
804
704
150
810
820
289
785
758
765
405
803
1092
1096
913
310
934
307
160
937
1069
718
413
794
200
780
105
306
574
840
1068
897
505
779
632
786
806
106
354
201
835
1073
825
122
284
808
146
110
499
162
795
99
443
639
113
610
507
155
294
762
118
1115
1107
1070
518
635
98
1119
169
651
1077
777
814
311
279
501
540
514
1081
1050
882
1030
561
212
813
312
121
578
434
782
833
368
793
253
1067
139
214
420
225
787
558
506
774
1037
1082
1114
1051
1094
1080
789
839
144
837
467
1126
151
784
120
486
778
869
616
180
255
1102
140
193
560
381
790
678
909
797
119
380
458
776
692
1172
796
800
248
773
575
483
783
526
666
1054
145
304
148
1084
290
302
867
438
750
444
100
108
908
305
243
446
562
760
931
167
158
424
719
101
1090
775
771
737
166
203
439
807
826
435
112
1071
601
1039
1022
325
292
893
815
572
772
1042
111
592
1060
161
309
168
276
159
770
104
125
1012
1011
1190
519
1014
1163
1160
1005
1004
1016
1013
269
1162
1137
1132
1174
907
748
941
1063
222
852
154
516
175
345
766
879
464
700
7